<a href="https://colab.research.google.com/github/yahyahabib65/AI-623/blob/main/AI_623_PA0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1

## 1. Baseline Setup

In [ ]:
# Import Libraries
import torchvision
import torch
from torchvision.models import resnet152
import torchvision.datasets as ds


# (a) Use a pre-trained ResNet-152 from PyTorch.
model = resnet152(pretrained=True)

# (b) Replace the final classification layer to match a smaller dataset such as CIFAR-10.
#TODO Replace fc with Smaller Dataset i.e CIFAR-10
# Getting CIFAR10 Classes
num_classes = ds.CIFAR10(root='./data', train=True, download=True).classes
in_features = model.fc.in_features
model.fc = torch.nn.Linear(in_features, len(num_classes))

# (c) Train only the classification head while freezing the rest of the backbone.
# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Unfreezes the classification head
for param in model.fc.parameters():
    param.requires_grad = True




/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


100%|██████████| 230M/230M [00:01<00:00, 189MB/s]
100%|██████████| 170M/170M [00:05<00:00, 31.1MB/s]


In [ ]:
# (d) Record training and validation performance for a few epochs. say 4 epochs
# Training and Testing validation
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as T

# Data
transform = T.Compose([T.Resize(224), T.ToTensor()])
trainset = ds.CIFAR10(root='./data', train=True, transform=transform, download=True)
testset  = ds.CIFAR10(root='./data', train=False, transform=transform, download=True)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader  = DataLoader(testset, batch_size=64, shuffle=False)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)

epochs = 4
for epoch in range(epochs):
    # ---- Training ----
    model.train()
    train_loss, correct = 0, 0
    print(f"Number of Samples: {len(trainset)} in epoch {epoch}")
    for x, y in trainloader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        correct += (out.argmax(1) == y).sum().item()

    train_acc = correct / len(trainset)

    # ---- Validation ----
    model.eval()
    val_loss, correct = 0, 0
    with torch.no_grad():
        for x, y in testloader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)

            val_loss += loss.item()
            correct += (out.argmax(1) == y).sum().item()

    val_acc = correct / len(testset)

    print(f"Epoch {epoch+1}: "
          f"Train Loss={train_loss:.3f}, Train Acc={train_acc:.3f}, "
          f"Val Loss={val_loss:.3f}, Val Acc={val_acc:.3f}")


Number of Samples: 50000 in epoch 0
Epoch 1: Train Loss=530.471, Train Acc=0.779, Val Loss=85.662, Val Acc=0.810
Number of Samples: 50000 in epoch 1
Epoch 2: Train Loss=396.706, Train Acc=0.826, Val Loss=76.921, Val Acc=0.834
Number of Samples: 50000 in epoch 2
Epoch 3: Train Loss=378.926, Train Acc=0.833, Val Loss=72.655, Val Acc=0.840
Number of Samples: 50000 in epoch 3
Epoch 4: Train Loss=364.738, Train Acc=0.840, Val Loss=71.140, Val Acc=0.841


- CIFAR10 has 50K training Images and 10K Testing Images
- ResNet Model has 60 Million Parameters
- Data is highly Prone to Overfitting

### It is observed in model regarding transferability that model performs fairly well which is due to the fact that:
- Prior Layers before final classification layer Learn Generic Features such as Shapes Textures contrast and edges
- And these features are reusable accross datasets
